# Merging a model on Arcee Cloud

In this notebook, you will learn how to merge a model on Arcee Cloud.

You can run this demo for free thanks to the Arcee free tier, which allows unlimited base merges.

The Arcee documentation is available at [docs.arcee.ai](https://docs.arcee.ai/deployment/start-deployment).

## Prerequisites

Please [sign up](https://app.arcee.ai/account/signup) to Arcee Cloud and create an [API key](https://docs.arcee.ai/getting-arcee-api-key/getting-arcee-api-key).

Then, please update the cell below with your API key. Remember to keep this key safe, and **DON'T COMMIT IT to one of your repositories**.

In [ ]:
%env ARCEE_API_KEY=YOUR_API_KEY

Create a new Python environment (optional but recommended) and install [arcee-python](https://github.com/arcee-ai/arcee-python).

In [ ]:
# Uncomment the next three lines to create a virtual environment
#!pip install -q virtualenv
#!virtualenv -q arcee-cloud
#!source arcee-cloud/bin/activate

%pip install -q arcee-py

In [ ]:
import arcee

## Merging a model

In Arcee Cloud, merging is based on the [`mergekit`](https://github.com/arcee-ai/mergekit) open-source library. The merging operation must be defined in a configuration file: models to merge, merging algorithm and parameters.

Here's an example where we merge two variants of the [Mistral-7B-v0.1](https://huggingface.co/mistralai/Mistral-7B-v0.1) model hosted on the Hugging Face hub:
* The instruction-fine tuned [Mistral-7B-Instruct-v0.2](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2),
* [BioMistral](https://huggingface.co/BioMistral/BioMistral-7B), tailored for the biomedical domain.

As we use both models in the same proportions, we would expect the merge to inherit the alignment of the first model and the domain knowledge of the second model.

You can find more examples of merging configurations in the [`mergekit`](https://github.com/arcee-ai/mergekit) documentation and at [docs.arcee.ai](https://docs.arcee.ai/merging/start-yaml-merge).

In [ ]:
%%writefile mistral_7b_merge.yaml
models:
  - model: mistralai/Mistral-7B-Instruct-v0.2
    parameters:
      density: 0.5
      weight: 0.5
  - model: BioMistral/BioMistral-7B
    parameters:
      density: 0.5
      weight: 0.5
merge_method: ties
base_model: mistralai/Mistral-7B-v0.1
parameters:
  normalize: false
  int8_mask: true
dtype: float16

We're now ready to launch the merging operation. We'll use the `mergekit_yaml()` API and simply pass the names of the merging operation and of the configuration file.

In [ ]:
merging_name = "Mistral 7B merge"
merging_yaml = "mistral_7b_merge.yaml"

In [ ]:
help(arcee.mergekit_yaml)

In [ ]:
response = arcee.mergekit_yaml(merging_name, merging_yaml)
print(response)

Let's wait for the merging operation to complete. It should only takes a few minutes.

The `merging_status` API lets us query the current state of the merging operation.

In [ ]:
help(arcee.merging_status)

In [ ]:
from time import sleep

while True:
    response = arcee.merging_status(merging_name)
    if response["processing_state"] != "completed":
        print("Merging is in progress. Waiting 30 seconds before checking again.")
        sleep(30)
    else:
        print(response)
        break

## Deploying our merged model

Once merging is complete, we can deploy and test the merged model. As part of the Arcee Cloud free tier, this is free of charge and the endpoint will be automatically shut down after 2 hours.

Deployment should take 5-7 minutes. Please see the model deployment sample notebook for details.

In [ ]:
deployment_name = "Mistral 7B merge deployment"

In [ ]:
response = arcee.start_deployment(deployment_name=deployment_name, merging=merging_name)

while True:
    response = arcee.deployment_status(deployment_name)
    if response["deployment_processing_state"] == "pending":
        print("Deployment is in progress. Waiting 30 seconds before checking again.")
        sleep(30)
    else:
        print(response)
        break

Once the model endpoint is up and running, we can prompt the model with a domain-specific question.

In [ ]:
query = "As a biomedical expert, you provide the most accurate and detailed answer possible. \
    Explain the role of enzymes in the digestive system and give two examples of molecular reactions catalyzed by enzymes."

response = arcee.generate(deployment_name=deployment_name, query=query)
print(response["text"])

## Stopping our deployment

When we're done working with our model, we should stop the deployment to save resources and avoid unwanted charges.

The `stop_deployment()` API only requires the deployment name.

In [ ]:
arcee.stop_deployment(deployment_name=deployment_name)
arcee.deployment_status(deployment_name)

This concludes the model merging demonstration. Thank you for your time!

If you'd like to know more about using Arcee Cloud in your organization, please visit the [Arcee website](https://www.arcee.ai), or contact [sales@arcee.ai](mailto:sales@arcee.ai).
